In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def call(f, *args, m=None):
    if m is None:
        s = "(" + ", ".join(map(repr, args)) + ")"
        m = "%s%s" % (f.__name__, s)
    print(m)
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        except Exception: 
            print(str(e))
        else:
            return rv
        
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def setdo(mode, n2_or_sp, o2=None):
    call(h.setdo, mode, n2_or_sp,o2)
    
def doman(n2, o2):
    setdo(1, n2, o2)
    
def doauto(sp):
    setdo(0, sp)
    
def dooff():
    setdo(2, 0, 0)
    
def phoff():
    setph(2, 0, 0)
    
def setph(mode, co2, base=None):
    call(h.setph, mode, co2, base)
    
def phauto(sp):
    setph(0, sp)
        
def endbatch():
    call(h.endbatch)

def wait2(s):
    pass
wait = wait1

def agauto(sp):
    call(h.setag, 0, sp)
    
def setmg(sp):
    call(h.setmg, 1, sp)

def pid1():
    call(h.setconfig, "DO", "O2_P_Gain_(%/DO%)", 1.5)
    call(h.setconfig, "DO", 'O2_I_Time_(min)', 120)
    call(h.setconfig, "DO", 'O2_D_Time_(min)', 0)
    call(h.setconfig, "DO", 'O2_Beta', 0)
    
    
    call(h.setconfig, "DO", "N2_P_Gain_(%/DO%)", -5)
    call(h.setconfig, "DO", 'N2_I_Time_(min)', 50)
    call(h.setconfig, "DO", 'N2_D_Time_(min)', 0)
    call(h.setconfig, "DO", 'N2_Beta', 0)
    

In [ ]:
hours = 3600
h = HelloApp('192.168.1.14')
wait = wait1
phoff()
phauto(7.2)
wait(6*hours)
startbatch("IE21 1.3")
phauto(7.5)
wait(6*hours)
endbatch()
startbatch("IE21 1.4")
phauto(7.0)
wait(6*hours)
endbatch()

setph(2, 0, 0)
setph(0, 7.2, None)
Sleeping 21600 seconds, ending at 06/06/2018 06:55:01 PM
batchrunning()
startbatch('IE21 1.3')
setph(0, 7.5, None)
Sleeping 21600 seconds, ending at 06/07/2018 12:55:07 AM


# 3L Analysis

In [10]:
h = HelloApp('192.168.1.14')
p=''

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+" " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [ ]:
for s in ("IE21 1.3", "IE21 1.4"):
    dl(s)

In [4]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
        
def analf(f):
    open_file(f)
    return analyze(f)

### pH Stuff

In [5]:
def analyze(fn):
    c1 = cells.Find("pHPV")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "24"
    mkch(c1,"pHPV")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
    for v in "pHPV", "pHCO2ActualRequest(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)

In [8]:
import os
files = [p+bn+" " + bt() + ".csv" for bn in ("IE21 1.3", "IE21 1.4")]
xl = Excel()
for file in files: wb = analf(os.path.abspath(file))